### Apache spark 1  


#### New Features

What features are we most excited about in Apache Spark 1.0? While there are dozens of new features in the release, we’d like to highlight three.

##### Spark SQL

The biggest single addition to Apache Spark 1.0 is Spark SQL, a new module that we’ve previously blogged about. Spark SQL offers integrated support for SQL queries alongside existing Spark code, making it seamless to write applications that load structured data (from sources like Hive and Parquet) and run advanced analytics or ETL. Spark SQL will also be the backend for future versions of Shark, providing a simpler, more agile, and optimized execution engine.

##### Management and Deployment

Apache Spark 1.0 also includes major improvements to management and deployment. It adds full support for the Hadoop/YARN security model, running seamlessly in secured Hadoop clusters. It also drastically simplifies job submission, allowing users to easily deploy the same application and on a single machine, a Spark cluster, EC2, Mesos, or YARN. Packaging and deploying a Spark app has never been easier!

##### Java 8 API

Spark’s Java API has been extended to support Java 8 lambda expressions, allowing much more concise programming for users of Java 8. Spark still supports Java 6 and 7 through the older API. 

#### SPARK RDD

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf

In [2]:
sc = SparkContext(master='local[1]', appName='first app', conf= SparkConf().set('spark.executor.memory', '1g'))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
rdd = sc.parallelize([1,2,3,4,5,6], numSlices=2)
rdd.collect()

[1, 2, 3, 4, 5, 6]

#### SQL context

In [4]:
from pyspark.sql import SQLContext

In [5]:
sqlctx = SQLContext(sc)

/home/audacious/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
# Load a text file and convert each line to a dictionary.
lines = sc.textFile("examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: {"name": p[0], "age": int(p[1])})

In [6]:
# Infer the schema, and register the SchemaRDD as a table.
# In future versions of PySpark we would like to add support for registering RDDs with other
# datatypes as tables
peopleTable = sqlCtx.inferSchema(people)
peopleTable.registerAsTable("people")


NameError: name 'sqlCtx' is not defined

In [ ]:
# SQL can be run over SchemaRDDs that have been registered as a table.
teenagers = sqlCtx.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are RDDs and support all the normal RDD operations.
teenNames = teenagers.map(lambda p: "Name: " + p.name)


#### Using Parquet

Parquet is a columnar format that is supported by many other data processing systems. Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data. Using the data from the above example:

In [ ]:
peopleTable # The SchemaRDD from the previous example.

# SchemaRDDs can be saved as Parquet files, maintaining the schema information.
peopleTable.saveAsParquetFile("people.parquet")

# Read in the Parquet file created above.  Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a SchemaRDD.
parquetFile = sqlCtx.parquetFile("people.parquet")

# Parquet files can also be registered as tables and then used in SQL statements.
parquetFile.registerAsTable("parquetFile");
teenagers = sqlCtx.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")



### Hive
When working with Hive one must construct a HiveContext, which inherits from SQLContext, and adds support for finding tables in in the MetaStore and writing queries using HiveQL. In addition to the sql method a HiveContext also provides an hql methods, which allows queries to be expressed in HiveQL.

In [ ]:
from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)

hiveCtx.hql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING)")
hiveCtx.hql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")

# Queries can be expressed in HiveQL.
results = hiveCtx.hql("FROM src SELECT key, value").collect()


In [25]:
#### Spark Streamming -- no support with python or pyspark introduced in spark 2

### Introducing Apache Spark 2.0

Today, we're excited to announce the general availability of Apache Spark 2.0 on Databricks. This release builds on what the community has learned in the past two years, doubling down on what users love and fixing the pain points. This post summarizes the three major themes—easier, faster, and smarter—that comprise Spark 2.0. We also explore many of them in more detail in our anthology of Spark 2.0 content.

Two months ago, we launched a preview release of Apache Spark 2.0 on Databricks. As you can see in the chart below, 10% of our clusters are already using this release, as customers experiment with the new features and give us feedback. Thanks to this experience, we are excited to be the first commercial vendor to support Spark 2.0. 

<img src='image00.png' >

Now, let’s dive into what's new in Apache Spark 2.0.
#### Easier: ANSI SQL and Streamlined APIs

One thing we are proud of in Spark is APIs that are simple, intuitive, and expressive. Spark 2.0 continues this tradition, focusing on two areas: (1) standard SQL support and (2) unifying DataFrame/Dataset API.

On the SQL side, we have significantly expanded Spark's SQL support, with the introduction of a new ANSI SQL parser and subqueries. Spark 2.0 can run all the 99 TPC-DS queries, which require many of the SQL:2003 features. Because SQL has been one of the primary interfaces to Spark, these extended capabilities drastically reduce the effort of porting legacy applications.

On the programmatic API side, we have streamlined Spark's APIs:

1. Unifying DataFrames and Datasets in Scala/Java: Starting in Spark 2.0, DataFrame is just a type alias for Dataset of Row. Both the typed methods (e.g. map, filter, groupByKey) and the untyped methods (e.g. select, groupBy) are available on the Dataset class. Also, this new combined Dataset interface is the abstraction used for Structured Streaming. Since compile-time type-safety is not a feature in Python and R, the concept of Dataset does not apply to these language APIs. Instead, DataFrame remains the primary interface there, and is analogous to the single-node data frame notion in these languages. Get a peek from this blog for the stories behind these APIs.
    
2. SparkSession: a new entry point that supersedes SQLContext and HiveContext. For users of the DataFrame API, a common source of confusion for Spark is which “context” to use. Note that the old SQLContext and HiveContext classes are still kept for backward compatibility.
    
3. Simpler, more performant Accumulator API: We have designed a new Accumulator API that has a simpler type hierarchy and support specialization for primitive types. The old Accumulator API has been deprecated but retained for backward compatibility
    
4. DataFrame-based Machine Learning API emerges as the primary ML API: With Spark 2.0, the spark.ml package, with its “pipeline” APIs, will emerge as the primary machine learning API. While the original spark.mllib package is preserved, future development will focus on the DataFrame-based API.
    
5. Machine learning pipeline persistence: Users can now save and load machine learning pipelines and models across all programming languages supported by Spark. See this blog post for more details.
    
6. Distributed algorithms in R: Added support for Generalized Linear Models (GLM), Naive Bayes, Survival Regression, and K-Means in R.
    
7. User-defined functions (UDFs) in R: Added support for running partition level UDFs (dapply and gapply) and hyper-parameter tuning (lapply).

#### Faster: Apache Spark as a Compiler

According to our 2015 Spark Survey, 91% of users consider performance as the most important aspect of Apache Spark. As a result, performance optimizations have always been a focus in our Spark development. Before we started planning our contributions to Spark 2.0, we asked ourselves a question: Spark is already pretty fast, but can we push the boundary and make Spark 10X faster?

This question led us to fundamentally rethink the way we build Spark’s physical execution layer. When you look into a modern data engine (e.g. Spark or other MPP databases), majority of the CPU cycles are spent in useless work, such as making virtual function calls or reading/writing intermediate data to CPU cache or memory. Optimizing performance by reducing the amount of CPU cycles wasted in these useless work has been a long time focus of modern compilers.

Spark 2.0 ships with the second generation Tungsten engine. This engine builds upon ideas from modern compilers and MPP databases and applies them to Spark workloads. The main idea is to emit optimized code at runtime that collapses the entire query into a single function, eliminating virtual function calls and leveraging CPU registers for intermediate data. We call this technique “whole-stage code generation.”

To give you a teaser, we have measured the time (in nanoseconds) it takes to process a row on one core for some of the operators in Spark 1.6 vs. Spark 2.0. The table below shows the improvements in Spark 2.0. Spark 1.6 also included an expression code generation technique that is used in some state-of-the-art commercial databases, but as you can see, many operators became an order of magnitude faster with whole-stage code generation. 


In [8]:
import pandas as pd

In [9]:
data = """primitive	Spark 1.6	Spark 2.0
filter	15ns	1.1ns
sum w/o group	14ns	0.9ns
sum w/ group	79ns	10.7ns
hash join	115ns	4.0ns
sort (8-bit entropy)	620ns	5.3ns
sort (64-bit entropy)	620ns	40ns
sort-merge join	750ns	700ns"""

In [18]:
pd.DataFrame([dat.split('\t') for dat in data.splitlines()])

,0,1,2
0,primitive,Spark 1.6,Spark 2.0
1,filter,15ns,1.1ns
2,sum w/o group,14ns,0.9ns
3,sum w/ group,79ns,10.7ns
4,hash join,115ns,4.0ns
5,sort (8-bit entropy),620ns,5.3ns
6,sort (64-bit entropy),620ns,40ns
7,sort-merge join,750ns,700ns


How does this new engine work on end-to-end queries? We did some preliminary analysis using TPC-DS queries to compare Spark 1.6 and Spark 2.0: 

<img src = 'preliminary-tpc-ds-spark-2-0-vs-1-6.png'>

Beyond whole-stage code generation to improve performance, a lot of work has also gone into improving the Catalyst optimizer for general query optimizations such as nullability propagation, as well as a new vectorized Parquet decoder that improved Parquet scan throughput by 3X. Read this blog post for more detail on the optimizations in Spark 2.0.
#### Smarter: Structured Streaming

Spark Streaming has long led the big data space as one of the first systems unifying batch and streaming computation. When its streaming API, called DStreams, was introduced in Spark 0.7, it offered developers with several powerful properties: exactly-once semantics, fault-tolerance at scale, strong consistency guarantees and high throughput.

However, after working with hundreds of real-world deployments of Spark Streaming, we found that applications that need to make decisions in real-time often require more than just a streaming engine. They require deep integration of the batch stack and the streaming stack, interaction with external storage systems, as well as the ability to cope with changes in business logic. As a result, enterprises want more than just a streaming engine; instead they need a full stack that enables them to develop end-to-end “continuous applications.”

Spark 2.0 tackles these use cases through a new API called Structured Streaming. Compared to existing streaming systems, Structured Streaming makes three key improvements:

1. Integrated API with batch jobs. To run a streaming computation, developers simply write a batch computation against the DataFrame / Dataset API, and Spark automatically incrementalizes the computation to run it in a streaming fashion (i.e. update the result as data comes in). This powerful design means that developers don't have to manually manage state, failures, or keeping the application in sync with batch jobs. Instead, the streaming job always gives the same answer as a batch job on the same data.
    
2. Transactional interaction with storage systems. Structured Streaming handles fault tolerance and consistency holistically across the engine and storage systems, making it easy to write applications that update a live database used for serving, join in static data, or move data reliably between storage systems.
    
3. Rich integration with the rest of Spark. Structured Streaming supports interactive queries on streaming data through Spark SQL, joins against static data, and many libraries that already use DataFrames, letting developers build complete applications instead of just streaming pipelines. In the future, expect more integrations with MLlib and other libraries.

Spark 2.0 ships with an initial, alpha version of Structured Streaming, as a (surprisingly small!) extension to the DataFrame/Dataset API. This makes it easy to adopt for existing Spark users that want to answer new questions in real-time. Other key features include support for event-time based processing, out-of-order/delayed data, interactive queries, and interaction with non-streaming data sources and sinks.

We also updated the Databricks workspace to support Structured Streaming. For example, when launching a streaming query, the notebook UI will automatically display its status.image01

<img src='image01.png'>



In [19]:
### RDD

In [22]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local[1]') \
            .config(conf=SparkConf().setAppName('sec app')) \
            .getOrCreate()

23/01/19 16:11:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [23]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6], numSlices=2)
rdd.collect()

[1, 2, 3, 4, 5, 6]

In [24]:
### sql and Hsql

In [ ]:
df = spark.read.csv('/user/people.csv', inferSchema=True, header=True)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

In [ ]:
# spark is an existing SparkSession
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")

# Queries are expressed in HiveQL
spark.sql("SELECT * FROM src").show()

In [ ]:
### Spark Structural Streamming

In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

### Spark 3.0 Highlights

In Spark 3.0, the whole community resolved more than 3,400 JIRAs. Spark SQL and the Core are the new core module, and all the other components are built on Spark SQL and the Core. Today, the pull requests for Spark SQL and the core constitute more than 60% of Spark 3.0. In the last few releases, the percentage keeps going up. Today, we will focus on the key features in both Spark SQL and the Core.

This release delivered many new capabilities, performance gains, and extended compatibility for the Spark ecosystem. This is a combination of the tremendous contributions from the open-source community. It is impossible to discuss the new features within 16 minutes. We resolved more than 3,400 JIRAs. Even in this light, I did my best, but I only can put 24 new Spark 3.0 features. Today, we would like to present some of them. First, let us talk about the performance-related features.
#### Spark 3.0 Enhanced Performance

High performance is one of the major advantages when people select Spark as their computation engine. This release keeps enhancing the performance for interactive, batch, streaming, and [inaudible] workloads. Here, I will first cover four of the performance features in SQL query compilers. Later, Wenchen will talk about the performance enhancement for building data sources.

The four major features in query compilers include a new framework for adaptive query execution and a new runtime filtering for dynamic partition pruning. And also, we greatly reduce the overhead of our query compiler by more than a half, especially on the optimizer overhead and the SQL cache synchronization. Supporting a complete set of join hints is another useful features many people are waiting for.

Adaptive query execution was available at the previous releases. However, the previous framework has a few major drawbacks. Very few companies are using it in the production systems. In this release, Databricks and the [inaudible] work together and redesigned the new framework and resolved all the known issues. Let us talk about what we did in this release.
#### Spark Catalyst Optimizer

Michael Armbrust. He is the creator of Spark SQL and also, Catalyst Optimizer. In the initial release of Spark SQL, all the optimizer rules are heuristic-based. To generate good query plans, the query optimizer needs to understand the data characteristics. Then in Spark 2.x, we introduced a cost-based optimizer. However, in most cases, data statistics are commonly absent, especially when statistics collection is even more expensive than the data processing in the [search?]. Even if the statistics are available, the statistics are likely out of date. Based on the storage and the compute separation in Spark, the characteristics of data [rival?] is unpredictable. The costs are often misestimated due to the different deployment environment and the black box user-defined functions. We are unable to estimate the cost for the UDF. Basically, in many cases, Spark optimizer is enabled to generate the best plan due to this limitation.

#### Adaptive Query Execution

For all these reasons, runtime adaptivity becomes more critical for Spark than the traditional systems. So this release introduced a new adaptive query execution framework called AQE. The basic idea of adaptive planning is simple. We optimize the execution plan using the existing rules of the optimizer and the planner after we collect more accurate statistics from the finished plans.

The red line shows the new logics we added in this release. Instead of directly optimizing the execution plans, we send back the unfinished plan segments and then use an existing optimizer and planner to optimize them at the end and build a new execution plan. This release includes three adaptive features. We can convert the soft merge join to broadcast hash join, based on the runtime statistics. We can shrink the number of reducers after over-partitioning. We can also handle the skew join at runtime. If you want to know more details, please read the blog post I posted here. Today, I will briefly explain them one by one.

Maybe most of you already learn many performance tuning tips. For example, to make your join faster, you might guide your optimizer to choose a broadcast hash join instead of the sort merge join. You can increase the spark.sql.autobroadcastjointhreshold or use a broadcast join hint. However, it is hard to tune it. You might hit out of memory exceptions and even get worse performance. Even if it works now, it is hard to maintain over time because it is sensitive to your data workloads.

You might be wondering why Spark is unable to make the wise choice by itself. I can easily list multiple reasons.

1. the statistics might be missing or out of date.
2. the file is compressed.
3. the file format is column-based, so the file size does not represent the actual data volume.
4. the filters could be compressed
5. (the filters) might also contain the black box UDFs.
6. The whole query fragments might be large, complex, and it is hard to estimate the actual data volume for Spark to make the best choice.

#### Convert Sort Merge Join to Broadcast Hash Join

So this is an example to show how AQE converts a sort merge join to a broadcast hash join at runtime. First, execute the leave stages. Query the statistics from the shuffle operators which materialize the query fragments. You can see the actual size of stage two is much smaller than the estimated size reduced from 30 megabytes to 8 megabytes so we can optimize the remaining plan and change the join algorithm from sort merge join to broadcast hash join.

Another popular performance tuning tip is to tune the configuration spark.sql.shuffle.partitions. The default value is a magic number, 200. Previously, the original default is 8. Later, it was increased to 200. I believe no one knows the reason why it become 200 instead of 50, 400, or 2,000. It is very hard to tune it, to be honest. Because it is a global configuration, it is almost impossible to decide the best value for every query’s fragment using a single configuration, especially when your query plan is huge and complex.

If you set it to very small values, the partition will be huge, and the aggregation and the sort might need to spew the data to the disk. If the configuration values are too big, the partition will be small. But the number of partitions is big. It will cause inefficient IO and the performance bottleneck could be the task scheduler. Then it will slow down everybody. Also, it is very hard to maintain over time.
#### Dynamically Coalesce Shuffle Partitions

Until you can solve it in a smart way, we can first increase our initial partition number to a big one. After we execute the leave query stage, we can know the actual size of each partition. Then we can automatically correlate the nearby partitions and automatically reduce the number of partitions to a smaller number. This example shows how we reduce the number of partitions from 50 to 5 at runtime. And we added the actual coalesce at runtime.
#### Data Skew

One more popular performance tuning tip is about data skew. Data skew is very annoying. You could see some long-running or frozen task and a lot of disks spinning and a very low resource authorization rate in most nodes and even out of memory. Our Spark community might tell you many different ways to solve such a typical performance problem. You can find the skew value and the right queries to handle the skew value separately. And also, you can add the actual skew keys that can remove the data skew, either new columns or some existing columns. Anyway, you have to manually rewrite your queries, and this is annoying and sensitive to your workloads, too, which could be changed over time.

This is an example without the skew optimization. Because of data skew, after the shuffle, the shuffle partition, A0, will be very large. If we do a join on these two tables, the whole performance bottleneck is to join the values for this specific partition, A0. For this partition, A0, the cost of shuffle, sort, and merge are much bigger than the other partitions. Everyone is waiting for the partition 0 to complete and slow down the execution of the whole query.

Our adaptive query execution can handle it very well in the data skew case. After executing the leaf stages (stages one and stage two), we can optimize our queries with a skew shuffle reader. Basically, it will split the skew partitions into smaller subpartitions after we realize some shuffle partitions are too big.

Let us use same example to show how to resolve it using adaptive query execution. After realizing partitions are too large, AQE will add a skew reader to automatically split table A’s partition part 0 to three segments: split 0, split 1, and split 2. Then it will also duplicate another side for table B. Then we will have three copies for table B’s part 0.

After this step, we can parallelize the shuffle reading, sorting, merging for this split partition A0. We can avoid generating very big partition for the sort merge join. Overall, it will be much faster.

Based on a terabyte of TPC-DS benchmark, without statistics, Spark 3.0 can make Q7 eight times faster and also achieve two times fast and speed up for Q5 and more than 1.1 speed up for another 26 queries. So this is just the beginning. In the future releases, we will continue to improve the compiler and introduce more new adaptive rules.

#### Dynamic Partition Pruning

The second performance features I want to highlight is dynamic partition pruning. So this is another runtime optimization rule. Basically, dynamic partition pruning is to avoid partition scanning based on the queried results of the other query fragments. It is important for star schema queries. We can achieve a significant speed up in TPC-DS queries.

So this is a number, in a TPC-DS benchmark, 60 out of 102 queries show a significant speed up between 2 times and 18 times. It is to prune the partitions that joins read from the fact table T1 by identifying those partitions that result from filtering the dimension table, T2.

Let us explain it step by step. First, we will do the filter push down in the left side. And on the right side, we can generate a new filter for the partition column PP because join P is a partition column. Then we get the query results of the left side. We can reuse our query results and generate the lists of constant values, EPP, and filter result. Now, we can push down the in filter in the right side. This will avoid scanning all the partitions of the huge fact table, T1. For this example, we can avoid scanning 90% of partitioning. With this dynamic partition pruning, we can achieve 33 times speed up.
#### JOIN Optimizer Hints

So the last performance feature is join hints. Join hints are very common optimizer hints. It can influence the optimizer to choose an expected join strategies. Previously, we already have a broadcast hash join. In this release, we also add the hints for the other three join strategies: sort merge join, shuffle hash join, and the shuffle nested loop join.

Please remember, this should be used very carefully. It is difficult to manage over time because it is sensitive to your workloads. If your workloads’ patterns are not stable, the hint could even make your query much slower.

Here are examples how to use these hints in the SQL queries. You also can do the same thing in the DataFrame API. When we decide the join strategies, [our leads are different here?].

1. So a broadcast hash join requires one side to be small, no shuffle, no sort, so it performs very fast.
    
2. For the shuffle hash join, it needs to shuffle the data but no sort is needed. So it can handle the large tables but will still hit out of memory if the data is skewed.
    
3. Sort merge join is much more robust. It can handle any data size. It needs to shuffle and salt data slower in most cases when the table size is small compared with a broadcast hash join.

4. And also, shuffle nested loop join, it doesn’t require the join keys, unlike the other three join strategies.

#### Richer APIs: new features and simplify development

To enable new use cases and simplify the Spark application development, this release delivers a new capability and enhanced interesting features.
#### Pandas UDF

Let’s, first, talk about Pandas UDF. This is a pretty popular performance features for the PySpark users.

So let us talk about the history of UDF support in PySpark. In the first release of Python support, 2013, we already support Python lambda functions for RDD API. Then in 2014, users can register Python UDF for Spark SQL. Starting from Spark 2.0, Python UDF registration is session-based. And then next year, users can register the use of Java UDF in Python API. In 2018, we introduced Pandas UDF. In this release, we redesigned the interface for Pandas UDF by using the Python tab hints and added more tabs for the Pandas UDFs.

To adjust our compatibility with the old Pandas UDFs from Apache Spark 2.0 with the Python 2.6 and above, Python [inaudible] such as pandas.Series, Pandas DataFrame, cube hole, and the iterator can be used to impress new Pandas UDF types. For example, in Spark 2.3, we have a Scala UDF. The input is a pandas.Series and its output is also pandas.Series. In Spark 2.0, we do not require users to remember any UDF types. You just need to specify the input and the output types. In Spark 2.3, we also have a Grouped Map Pandas UDF, so input is a Pandas DataFrame, and the output is also Pandas DataFrames.
#### Old vs New Pandas UDF interface

This slide shows the difference between the old and the new interface. The same here. The new interface can also be used for the existing Grouped Aggregate Pandas UDFs. In addition, the old Pandas UDF was split into two API categories: Pandas UDFs and Pandas function APIs. You can treat Pandas UDFs in the same way that you use the other PySpark column instance.

For example, here, calculate the values. You are calling the Pandas UDF calculate. We do support the new Pandas UDF types from iterators of series to iterator other series and from iterators of multiple series to iterator of series. So this is useful for [inaudible] state initialization of your Pandas UDFs and also useful for Pandas UDF parquet.

However, you can now use Pandas function APIs with this column instance. Here are these two examples: map Pandas function API and the core group, the map Pandas UDF, the APIs. These APIs are newly added in these units.

#### Accelerator-aware Scheduling

I will start with, straight away, our scheduler. In 2018 Spark Summit, we already announced the new project [inaudible]. As you’re now aware, our scheduler is part of this project. It can be widely used for executing special workloads. In this release, we support standalone, YARN, and Kubernetes scheduler scheduler backend. So far, users need to specify the require resources using a [inaudible] configs.

In the future, we will support the job, stage, and task levels. To further understand this feature, let’s look at the workflow. Ideally, the cost manager should be able to automatically discover resources, like GPUs. When the user submits an application with resource request, Spark should pass the resources request to a cluster manager and then the cluster manager cooperates to allocate and launch executors with the required resources. After Spark job is submitted, Spark should schedule tasks on available executors, and the cluster manager should track the results usage and perform dynamic resource allocation.

For example, when there are too many pending tasks, the cluster manager should allocate more executors to run more tasks at the same time. When a task is running, the user shall be able to retrieve the assigned resources and use them in their code. In the meanwhile, cluster manager shall monitor and recover failed executions. Now, let’s look at how can a cluster manager discover resources and how can users request resources.

As an admin of the cluster, I can specify a script to auto discover executors. The discovery script can be specified separately on Java as executors. We also provided an example to auto discover Nvidia GPU resources. You can adjust this example script for other kinds of resources. Then as a user of Spark, I can request resources at the application level. I can use the config spark.executor.resource.{resourceName}.amount and the corresponding config for Java to specify the executors amount on the Java and executors. Also, I can use the config spark.task.resource.{resourceName}.amount to specify the executors required by each task. As I mentioned earlier, we will support more time-proven labor later, like the job or stage labor. Please stay tuned.
#### Retrieve Assigned Accelerators

Next, we’ll see how you can leverage the assigned executors to actually execute your workloads, which is probably the most important part to the users. So as a user of Spark, I can retrieve the assigned executors from the task content. Here is an example in PySpark. The contents of resources returns a map from the resource name to resource info. In the example, we request for GPUs, and we can take the GPU address from the resource map. Then we launch the TensorFlow to train my model within GPUs. Spark will take care of the resource allocation and acceleration and also monitor the executors here for failure recovery, which makes my life much easier.
#### Cluster Manager Support

As I mentioned earlier, the executor aware of scheduling support has been added to standalone, YARN, and Kubernetes cost manager. You can check the Spark JIRA tickets to see more details. Unfortunately, the Mesos support is still not available. We’d really appreciate it if any Mesos expert has interest and is willing to help the Spark community to add the Mesos support. Please leave a comment in the [inaudible] if you want to work on it. Thanks in advance.
#### Improved Spark Web UI for Accelerators

Last but not the least, we also improved the Spark Web UI to show all the discovery resources on the executor page. In this page, we can see that there are GPUs available on the executor one. You can check the Web UI to see how many executors are available in the cluster, so you can better schedule your jobs.
#### 32 New Built-in Functions

In this release, we also introduced 32 new built-in functions and add high auto functions in the Scalar API. The Spark community pays a lot of attention to compatibility. We have investigated many other ecosystems, like the PostgreSQL, and implemented many commonly used functions in Spark.

Hopefully, these new built-in functions can make it faster to build your queries as you don’t need to waste time to learn a lot of UDFs.

Due to the time limitations, I can’t go over all the functions here, so let me just introduce some map type functions as an example.

When you deal with map type values, it’s common to get the keys and values for the map as an array. There are two functions, map keys and map values can do this for you. The example is from the Databricks runtime notebook. Or you may want to do something more complicated, like creating a new map by transforming the original map where it’s a keys and a map values functions. So if there are two functions, transform keys and transform values can do this for you, and you just need to write a handler function to specify the transformation logic.

As I mentioned earlier, the functions also have Scalar APIs rather than the SQL API. Here is an example about how to do the same thing, but it’s a Scalar API. You can just write a normal Scala function, which takes the [kernel?] objects as the input to have the same effect as the SQL API.
#### Monitoring and Debuggability

This release also includes many enhancements and makes the monitoring more comprehensive and stable. We can make it easier to close out and get back to your Spark applications.
#### Structured Streaming UI

The first feature I will talk to you about is the new UI for the Spark streaming. Here, the drive to show it– Spark streaming was initially introduced in Spark 2.0. This release has the dedicated– it was Spark web UI for inspection of these streaming jobs. This UI offers two sets of statistics: one, abbreviate information of [completed?] streaming queries and two, detailed statistics information about the streaming query including the input rate, processor rate, input loads, [inaudible], operation duration and others. More specifically, the input rate and processor rate means how many records per second the streaming software produces and the Spark streaming engine processes. It can give you a sense about if the streaming engine is fast enough to process the continuous input data. Similarly, you can tell it from the past duration as well. If many batch takes more time than the micro-batch [inaudible], it means the engine is not fast enough to process your data, and you may need to enable the [inaudible] feature to make the source produce the data slower.

And so operating time is also a very useful matrix. It tells you the time spent on each operator so that you can know where is the bottleneck in your query.
#### DDL and DML enhancements

We also have many different enhancements in DDL and DML commands. Let me talk about the improvements in the EXPLAIN command as an example. This is a typical output of the EXPLAIN command. You have many operators in the query plan tree and some operators have other additional information. Reading plans is critical for understanding and attuning queries. The existing solution looks [inaudible], and, as a stream of each operator, can be very wide or even truncated. And it becomes wider and wider each release as we add more and more information in the operator to help debugging.

This release, we enhance the EXPLAIN command with a new formatted mode and also provided a capability to dump the plans to the files. You can see it becomes much easier to read and understand. So here is a very simple plan tree at the beginning. Then follows a detailed section for each operator. This makes it very easy to get an overview of the query by looking at the plan tree. It also makes it very easy to see the details of each operator as the information is now stacked vertically. And in the end, there is a section to show all the subqueries. In the future releases, we will add more and more useful information for each operator.

This release, we also introduced a new API to define your own metrics to observe data quality. Data quality is very important to many applications. It’s usually easy to define metrics for data quality by some [other?] function, for example, but it’s also hard to calculate the metrics, especially for streaming queries.

For example, you want to keep monitoring the data quality of your streaming source. You can simply define the metrics as the percentage of the error records. Then you can do two things. Make it a habit. One, code observe method of the streaming error rate to define your metrics with a name and the start of stream. So this example, the name is data quality and the matrix, it just will count the error record and see how many percent of it in the total lookups.

Two, you add a listener to watch the streaming process events, and in the case of your matrix, the name, do whatever you want to do, such as sending an email if there are more than 5% error data.


#### SQL Compatibility

Now, let’s move to the next topic. SQL compatibility is also super critical for workloads mapped from the other database systems through Spark SQL. In this release, we introduced the ANSI store assignment policy for table insertion. We added runtime overall checking with respect to ANSI results keywords into the parser. We also switched the calendar to the widely-used calendar which is the ISO and SQL standard.

Let’s look at how the first two features can help you to enforce data quality. I say more about the assignment. It’s something like assigning a value to a variable in programming language. In the SQL world, it is table insertion or upsert, which is kind of assigning values to a table column.

Now, let’s see an example. Assume there is a table with two columns, I and J, which are type int and type string. If we write a int value to the string column, it’s totally okay. It’s totally safe. However, if we write a string value to the int column, it’s risky. The string value is very likely to not be in integer form, and Spark will fail and worry about it.

If you do believe your string values are safe to be inserted into an int column, you can add a cast manually to bypass the type check in Spark.

We can also write long type of values to the int column, and Spark will do the overflow check at runtime. If your input data is invalid, Spark will be show exception at runtime to tell you about it. In this example, the integer one is okay, but the larger value below can’t fit the integer type, and you’ll receive this error if you run this table insertion command which tells you about the overflow problem.
#### Built-in Data Source Enhancements

Also, this release enhances built-in data sources. For example, to populate data source, we can’t do nested column and filter pushdown. Also, we support [inaudible] for CSV files. This release also introduced a new [inaudible] resource and also a new [inaudible] resource for testing and benchmarking. Let me further introduce the origin of nested columns in Parquet and ORC resource.

The first one is a kind of [baloney?]. [inaudible] like [inaudible] and [ORC?], we can skip reading some [inaudible] in the blocks if they don’t contain the columns we need. This [technique?] can be applied to nested columns as well in Spark 2.0. To check if your query– in Spark 3.0. To check if your query can benefit from this [inaudible] or not, you can run the EXPLAIN command and see if the read schema over the file scan note strips the [inaudible] nested columns. In this example, only the nested column is inserted, so the read schema only contains A.

[inaudible] is also a very popular technical [inaudible]. Similarly, you can also check the expand result and see if the [pushed?] filters over the file scan note contains the name of column filters. In this example, we do have a filter where it’s nested column A, and it does appear in the [pushed?] filter, which makes this version happen in this query.
#### Catalog plugin API

This release also expands other efforts on the extensibility and ecosystem like the v2 API enhancements, Java 11, Hadoop, and [inaudible] support. [inaudible] API. This release extends the [inaudible] to API by adding the Catalog plugin. The Catalog plug-in API allows users to reject their own [inaudible] and take over the [inaudible] data operations from Spark. This can give end users a more seamless experience to assist external tables. Now, end users [inaudible] reject [inaudible] and manipulate the tables [inaudible], where before, end users have to reject each table. For example, let’s say you have rejected a MySQL connector [inaudible] named MySQL. You can use SELECT to get data from existing MySQL table. We can also INSERT into a MySQL table with Spark’s [inaudible]. You can also create an outer tables in MySQL with Spark, which was just not possible before, because before, we don’t have the Catalog plug-in. Now this example will be available in Spark 3.1 when we finish [inaudible].
#### When to use Data Source V2 API?

Some people may have a question. Now Spark has both – it has a V1 and a V2 APIs – which one should I use? In general, we want everyone to move to V2 [inaudible]. But the V2 API is not ready yet as we need more feedback to polish the API. Here are some tips about when to pick the V2 API. So if you want [inaudible], the catalogue function it is, so it has to be the V2 because V1 API doesn’t have this ability. If you want to support both versions streaming in your data source, then you should use V2 because in V1, the streaming and the [inaudible] are different APIs which makes it harder to reuse the code. And if you are sensitive to the scan performance, then you can try the V2 API because it allows you to report the data provisioning to [inaudible] in Spark, and also it allows you to implement [inaudible] reader for better performance.

If you don’t care about this stuff and just want to [inaudible] source once and you change it, please use the V1 as V2 is not very stable.
#### Extensibility and Ecosystem

The ecosystem also evolves very fast. In this release, Spark can be better integrated into the ecosystem by supporting the newer version of these common components like Java 11, Hadoop 3, Hadoop 3 [inaudible], and Hadoop 2.3 [inaudible]. I want to mention some breaking changes here.

Starting from this release, we’re only building Spark with Scala 2.12, so Scala 2.11 is no longer [inaudible]. And we deprecated Python 2 too because it is end of life. In the download image, we put a build of Spark with different [inaudible] and Hadoop combinations. By default, it will be Hadoop 2.7 and it would have 2.3 [exclusion?]. There are another two [companies?] of previews available. One is Hadoop 2.7 and Hadoop 1.2 execution, which is for people who can’t upgrade their end forms. The other is Hadoop 3.2 and Hadoop 2.3 execution, which is for people who want to try Hadoop 3. We also extend the support for different Hadoop and Hive versions from 0.12 to 3.1.
#### Documentation Improvements

Documentation improvements is the last existing news I want to share with everyone. How to read on a standard the web UI is a common question to many new Spark users. This is especially true for Spark SQL users and Spark streaming users. They are using the [inaudible]. They usually don’t know what it is, and what our jobs [inaudible]. Also, the [inaudible] are using many queries and matrix names, which are not very clear to many users. Starting from this release, we add a new section for [inaudible] reading the web UI. It includes the [inaudible] job page and [inaudible] and also SQL streaming [inaudible]. This is just a start. We will continue to enhance it, then SQL reference.

Finally, this release already has a SQL reference for Spark SQL. Spark SQL is the most popular and important component in Spark. However, we did not have our own SQL reference to define the SQL [semantic?] and detailed behaviors. Let me quickly go over the major chapters in SQL reference. So we have a page to explain the ANSI components of Spark. So as I mentioned before, we have SQL compatibility, but to avoid [correcting?] the [effecting?] queries, we make it optional. So you can only enable the ANSI compatibility by enabling this flag.

You also have a page to explain the detailed semantic of each [inaudible], so you can know what it means and what’s the behavior of them. You also have a page to explain the data and partner strings used for formatting and parsing functions [inaudible]. There’s also a page to give the document for each function in Spark. We also have a page to explain the syntax, how to define the table or function [inaudible]. Also, there’s a page to explain the syntax and the semantics of each [inaudible] in Spark SQL.

Also, there’s a page to explain the null semantic. The null is a very special value in Spark SQL and other ecosystems. So there must be a page to either explain what’s the meaning of null in the null queries. Also, we have a page to explain the syntax for all the commands, like DDL and DML commands, and also insert is also included in the document. In fact, SELECT has so many features, so we want to have a page to explain all of them. Yeah, there are also a page for other special commands like SHOW TABLES.

Finally, [inaudible]. This is another critical enhancements in Spark 3.0 document. In this release, all the components have [inaudible] guides. When you upgrade your Spark version, you can read them carefully, and, in fact, [inaudible]. You might be wondering why it is much longer than the previous version. It’s because we try to document all the important looking changes you want to hear. If you upgrade into some errors, that’s another wordy document or slightly confusing error message, please open a ticket, and we will try and fix it in subsequent releases.

Now, that Spark is almost 10 years old now. The Spark community is very serious about making change. And we try our best to avoid [inaudible] changing. If you upgrade to Spark 3.0 at this time, you may see explicit error messages about changing. So the error message also provides config names for you to either go back to existing behavior or go with the new behavior.

this talk, we talked about many exciting features and improvements in Spark 3.0. Due to the lack of time, there are still many other nice features not being covered by this talk. Please download Spark 3.0 and try yourself.

You can also try the [inaudible] Databricks [inaudible] 10.0 beta. All the new features are already available. The Community Edition is for free. Without the contributions by the whole community, it is impossible to deliver such a successful release. It’s thanks to all the Spark committers all over the world. Thank you. Thank you, everyone. 

### RDD

In [27]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('appName').setMaster('local[1]')
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=first app, master=local[1]) created by __init__ at /tmp/ipykernel_53431/2806237059.py:1 

In [28]:
rdd = sc.parallelize([12,3,4,5],numSlices=2)
rdd.collect()

[12, 3, 4, 5]

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local[1]') \
            .config(conf=SparkConf().setAppName('sec app')) \
            .getOrCreate()

In [ ]:
df = spark.read.csv('/user/people.csv', inferSchema=True, header=True)

# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

In [ ]:
# spark is an existing SparkSession
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH 'examples/src/main/resources/kv1.txt' INTO TABLE src")

# Queries are expressed in HiveQL
spark.sql("SELECT * FROM src").show()

In [ ]:
## Spark Structural streamming

In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()